# Méthodes d'Ensembles


In [1]:
import pandas as pd
from prepdata import *
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import numpy as np

In [10]:
# Récupérer les données du dataset abalone8
X, y = data_recovery("abalone8")

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print(np.sum(X))
print(np.sum(y))

143072.78194
53


In [3]:
# Initialisation de StratifiedKFold
skf = StratifiedKFold(n_splits=5)

# Liste de différents C (lambda = 1 / C) à tester
C_values = [0.01, 0.1, 1.0, 10.0, 100.0]

# Stocker les résultats d'accuracy pour chaque valeur de C
best_C = None
best_accuracy = 0
C_accuracy_scores = {}

# Parcours de chaque valeur de C
for C in C_values:
    print(f"Testing SVM with C = {C}")
    
    accuracy_scores = []  # Stocker les scores d'accuracy pour chaque fold
    
    # Parcours des folds pour la cross-validation
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
        
        # Normalisation sur les k-1 folds (uniquement les données d'entraînement)
        scaler = Normalizer()
        X_train_fold = scaler.fit_transform(X_train_fold)
        
        # Normaliser les données du fold de validation avec les paramètres du train_fold
        X_val_fold = scaler.transform(X_val_fold)
        
        # Initialiser le modèle SVM avec kernel linéaire pour cette valeur de C
        clf = SVC(C=C, kernel='linear')
        
        # Entraînement du modèle sur le fold courant
        clf.fit(X_train_fold, y_train_fold)
        
        # Prédiction sur les données de validation
        y_pred_fold = clf.predict(X_val_fold)
        
        # Calcul de l'accuracy sur le fold de validation
        accuracy = accuracy_score(y_val_fold, y_pred_fold)
        accuracy_scores.append(accuracy)

    # Calcul de l'accuracy moyenne pour cette valeur de C
    mean_accuracy = np.mean(accuracy_scores)
    C_accuracy_scores[C] = mean_accuracy
    
    print(f"Mean cross-validated accuracy for C={C}: {mean_accuracy:.4f}")
    
    # Mettre à jour le meilleur C en fonction de l'accuracy
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_C = C

# Afficher le meilleur C et son accuracy
print(f"\nBest C: {best_C} with cross-validated accuracy: {best_accuracy:.4f}")

Testing SVM with C = 0.01
Mean cross-validated accuracy for C=0.01: 0.9672
Testing SVM with C = 0.1
Mean cross-validated accuracy for C=0.1: 0.9672
Testing SVM with C = 1.0
Mean cross-validated accuracy for C=1.0: 0.9672
Testing SVM with C = 10.0
Mean cross-validated accuracy for C=10.0: 0.9672
Testing SVM with C = 100.0
Mean cross-validated accuracy for C=100.0: 0.9672

Best C: 0.01 with cross-validated accuracy: 0.9672


In [4]:
# Entraînement du modèle final avec le meilleur C sur l'ensemble complet d'entraînement
scaler = Normalizer()  # Normalisation pour le modèle final
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf_final = SVC(C=best_C, kernel='linear')
clf_final.fit(X_train, y_train)

# Prédiction finale sur l'ensemble de test
y_test_pred = clf_final.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test accuracy with best C={best_C}: {test_accuracy:.4f}")

Test accuracy with best C=0.01: 0.9656


In [5]:
# Affichage de la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[309   0]
 [ 11   0]]
